In [1]:
import pandas as pd
outcome_analyses=pd.read_csv('outcome_analyses.csv',usecols=['outcome_analysis_id', 'nct_id', 'outcome_id','p_value'])
print(outcome_analyses.head(2))
print(outcome_analyses.nct_id.nunique())
print(outcome_analyses.shape)

   outcome_analysis_id       nct_id  outcome_id  p_value
0             35794071  NCT00798707    70015184    0.452
1             35794072  NCT00798707    70015184    0.016
24025
(292939, 4)


In [2]:
outcomes=pd.read_csv('outcomes.csv',usecols=['outcome_id','outcome_type'])
print(outcomes.head(2))
print(outcomes.shape)

   outcome_id         outcome_type
0    69958276              PRIMARY
1    69958277  OTHER_PRE_SPECIFIED
(579892, 2)


In [3]:
df_comb = outcome_analyses.merge(
    outcomes,
    on='outcome_id',
    how='left',
    validate='many_to_one'  
)
df_comb.head(5)

,outcome_analysis_id,nct_id,outcome_id,p_value,outcome_type
0,35794071,NCT00798707,70015184,0.452,PRIMARY
1,35794072,NCT00798707,70015184,0.016,PRIMARY
2,35794073,NCT00798707,70015185,0.160,SECONDARY
3,35794074,NCT00798707,70015185,0.028,SECONDARY
4,35735797,NCT02075476,69896008,0.048,PRIMARY


In [4]:
outcomes_prim=df_comb[df_comb.outcome_type=='PRIMARY']

print(outcomes_prim.head(5))
print(outcomes_prim.nct_id.nunique())
print(outcomes_prim.shape)

    outcome_analysis_id       nct_id  outcome_id  p_value outcome_type
0              35794071  NCT00798707    70015184   0.4520      PRIMARY
1              35794072  NCT00798707    70015184   0.0160      PRIMARY
4              35735797  NCT02075476    69896008   0.0480      PRIMARY
15             35852921  NCT02815267    70131019   0.0083      PRIMARY
16             35852922  NCT02815267    70131020   0.1805      PRIMARY
22582
(76458, 5)


In [5]:
def result_label(x):
    if x.isna().all():
        return 'unknown'
    return 'success' if (x <= 0.05).any() else 'fail'

study_results = outcomes_prim.groupby('nct_id')['p_value'].apply(result_label).reset_index(name='study_result')
print(study_results.head(5))
print(study_results.nct_id.nunique())
print(study_results.shape)
print(study_results.study_result.value_counts())

        nct_id study_result
0  NCT00000378      success
1  NCT00000392         fail
2  NCT00000620         fail
3  NCT00001656      success
4  NCT00001723      success
22582
(22582, 2)
study_result
success    12108
fail        6781
unknown     3693
Name: count, dtype: int64


In [6]:
study_results.to_csv('CTG_PVAL.csv', index=False)

## Merging with CTG dataset

In [8]:
df=pd.read_csv('CTG_Extra_features.csv')

In [9]:
study_results.rename(columns={
    'nct_id':'NCT_Number',
    'study_result':'study_success'},inplace=True)

In [11]:
# merge with main df
df_merged = df.merge(
    study_results,
    on='NCT_Number',
    how='left'
)
df_merged.shape

(164296, 71)

In [13]:
df_merged.to_csv('ctgALL_Extra_features0206_pval.csv', index=False)